<a href="https://colab.research.google.com/github/preetamjumech/SQL-Assignment_1/blob/main/Preetam_Saha_Roll_No_C22018_RDWH_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial based on [SQLCourse2](https://www.sqlcourse2.com/) <br>
Data is in this [Google Sheet](https://docs.google.com/spreadsheets/d/1m_LE_ahVHY0PrXmtyVVjO5CyqvmOMOwGT6hPYBYHBqw/edit?usp=sharing) <br>
The data needs to be downloaded as two CSV files from the Google Docs above <br>
This is a slightly simplified, local DB version of the [full exercise](https://github.com/Praxis-QR/RDWH/blob/main/SQLCourse2_Setup_with_LOAD_command.ipynb) that demonstrates both local and remote databases

#Using Local Server

## Install MySQL

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null
!/etc/init.d/mysql restart






 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


In [2]:
#Test the success of the installation with a the native, Command Line Interface (CLI) client that comes bundled with the mysql server. 
#Later on we will install the Python client
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


## Create Tables

In [3]:
!mysql praxisDB -e 'drop table if exists itemsordered'
# --------------------------------------------------------------
!mysql praxisDB -e '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int(4), price decimal(5,2)) \
'

In [4]:
!mysql praxisDB -e 'desc itemsordered'

+------------+--------------+------+-----+---------+-------+
| Field      | Type         | Null | Key | Default | Extra |
+------------+--------------+------+-----+---------+-------+
| customerid | char(5)      | YES  |     | NULL    |       |
| orderdate  | date         | YES  |     | NULL    |       |
| item       | varchar(30)  | YES  |     | NULL    |       |
| quantity   | int(4)       | YES  |     | NULL    |       |
| price      | decimal(5,2) | YES  |     | NULL    |       |
+------------+--------------+------+-----+---------+-------+


In [5]:
!mysql praxisDB -e 'drop table if exists customers'
# --------------------------------------------------------------
!mysql praxisDB -e '\
CREATE TABLE IF NOT EXISTS customers (customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'

In [6]:
!mysql praxisDB -e 'desc customers'

+------------+-------------+------+-----+---------+-------+
| Field      | Type        | Null | Key | Default | Extra |
+------------+-------------+------+-----+---------+-------+
| customerid | char(5)     | YES  |     | NULL    |       |
| firstname  | varchar(20) | YES  |     | NULL    |       |
| lastname   | varchar(20) | YES  |     | NULL    |       |
| city       | varchar(20) | YES  |     | NULL    |       |
| state      | varchar(20) | YES  |     | NULL    |       |
+------------+-------------+------+-----+---------+-------+


## Load Tables

In [7]:
# Download two CSV files from G-Drive into the Colab VM
!gdown https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
!gdown https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld

Downloading...
From: https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
To: /content/SQLCourse2 - ItemsOrdered.csv
100% 1.16k/1.16k [00:00<00:00, 1.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld
To: /content/SQLCourse2 - Customers.csv
100% 695/695 [00:00<00:00, 1.04MB/s]


In [8]:
# upload the two CSV files into the Colab VM
!ls

 sample_data  'SQLCourse2 - Customers.csv'  'SQLCourse2 - ItemsOrdered.csv'


In [9]:
# Note first line, with header columns ignored
# Note date is converted from string to proper date format
#
!mysql praxisDB -e "   \
LOAD DATA LOCAL INFILE 'SQLCourse2 - ItemsOrdered.csv' \
ignore \
INTO  TABLE itemsordered  \
FIELDS TERMINATED BY ',' enclosed by '\"' LINES TERMINATED BY '\r\n' IGNORE 1 LINES  \
(customerid,@orderdate, item, quantity, price) \
set orderdate = STR_TO_DATE(@orderdate,'%d-%b-%Y');\
"

In [10]:
#!mysql praxisDB -e 'select * from itemsordered;'

In [11]:
# Note first line, with header columns ignored
#
!mysql praxisDB -e "   \
LOAD DATA LOCAL INFILE 'SQLCourse2 - Customers.csv' INTO TABLE customers \
FIELDS TERMINATED BY ',' enclosed by '\"' LINES TERMINATED BY '\r\n'  IGNORE 1 LINES;  \
"

In [ ]:
#!mysql praxisDB -e 'select * from customers;'

# Pythons Functions for SQL commands

In [12]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [13]:
#To run any non-SELECT SQL command
def runCMD (loc, DDL):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (loc, CMD):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

## Test SQL calls from Python

In [14]:
# Set the location to whichever Database you need to access
#
# --------------------------------
#
location = 'localDB'
#location = 'remoteDB'

In [15]:
#runSELECT(location,'select * from customers')
runSELECT(location,'select * from itemsordered limit 5')

,customerid,orderdate,item,quantity,price
0,10330,1999-06-30,Pogo stick,1,28.00
1,10101,1999-06-30,Raft,1,58.00
2,10298,1999-07-01,Skateboard,1,33.00
3,10101,1999-07-01,Life Vest,4,125.00
4,10299,1999-07-06,Parachute,1,999.99


In [16]:
runSELECT(location,'select * from customers limit 5')

,customerid,firstname,lastname,city,state
0,10101,John,Gray,Lynden,Washington
1,10298,Leroy,Brown,Pinetop,Arizona
2,10299,Elroy,Keller,Snoqualmie,Washington
3,10315,Lisa,Jones,Oshkosh,Wisconsin
4,10325,Ginger,Schultz,Pocatello,Idaho


#Assignment 
Please go through each section of the following [exercise](https://www.sqlcourse.com/advanced-course/). Each SQL for a review question should be in a separate code cell and multiple code cells from each section should be grouped under one text cell. As shown below ...

##SELECT statement

In [17]:
#code
runSELECT(location,'select customerid,item,price from itemsordered where customerid = 10449')

,customerid,item,price
0,10449,Unicycle,180.79
1,10449,Snow Shoes,45.00
2,10449,Bicycle,380.50
3,10449,Canoe,280.00
4,10449,Flashlight,4.50
5,10449,Canoe paddle,40.00


In [18]:
#code
runSELECT(location,'select * from itemsordered where item = "Tent"')

,customerid,orderdate,item,quantity,price
0,10439,1999-09-18,Tent,1,88.00
1,10438,2000-01-18,Tent,1,79.99


In [19]:
#code
runSELECT(location,'select customerid,orderdate,item from itemsordered where item like "S%"')

,customerid,orderdate,item
0,10298,1999-07-01,Skateboard
1,10439,1999-08-14,Ski Poles
2,10449,1999-09-01,Snow Shoes
3,10410,1999-10-28,Sleeping Bag
4,10101,2000-03-08,Sleeping Bag
5,10330,2000-04-19,Shovel


In [20]:
#code
runSELECT(location,'select distinct item from itemsordered')

,item
0,Pogo stick
1,Raft
2,Skateboard
3,Life Vest
4,Parachute
5,Umbrella
6,Unicycle
7,Ski Poles
8,Rain Coat
9,Snow Shoes


##Aggregate functions

In [21]:
#code
runSELECT(location,'select max(price) from itemsordered')

,max(price)
0,999.99


In [22]:
#code
runSELECT(location,'select avg(price) from itemsordered where orderdate like "%12%" ')

,avg(price)
0,174.3125


In [23]:
#code
runSELECT(location,'select count(*) from itemsordered')

,count(*)
0,32


In [24]:
#code
runSELECT(location,'select min(price) from itemsordered where item = "Tent" ')

,min(price)
0,79.99


#Group by

In [25]:
#code
runSELECT(location,'select state, count(state) from customers group by state ')

,state,count(state)
0,Arizona,6
1,Colorado,2
2,Hawaii,1
3,Idaho,1
4,North Carolina,1
5,Oregon,2
6,South Carolina,1
7,Washington,2
8,Wisconsin,1


In [26]:
#code
runSELECT(location,'select item, max(price),min(price) from itemsordered group by item')

,item,max(price),min(price)
0,Bicycle,380.50,380.50
1,Canoe,280.00,280.00
2,Canoe paddle,40.00,40.00
3,Compass,8.00,8.00
4,Ear Muffs,12.50,12.50
5,Flashlight,28.00,4.50
6,Helmet,22.00,22.00
7,Hoola Hoop,14.75,14.75
8,Inflatable Mattress,38.00,38.00
9,Lantern,29.00,16.00


In [27]:
#code
runSELECT(location,'select customerid,count(customerid),sum(price) from itemsordered group by customerid')

,customerid,count(customerid),sum(price)
0,10101,6,320.75
1,10298,5,118.88
2,10299,2,1037.99
3,10315,1,8.00
4,10330,3,72.75
5,10339,1,4.50
6,10410,2,281.72
7,10413,1,32.00
8,10438,3,95.24
9,10439,2,113.50


#Having Clause

In [28]:
#code
runSELECT(location,'select state,count(state) from customers group by state having count(state)>1')

,state,count(state)
0,Arizona,6
1,Colorado,2
2,Oregon,2
3,Washington,2


In [29]:
#code
runSELECT(location,'select item,max(price),min(price) from itemsordered group by item having max(price)>190.00')

,item,max(price),min(price)
0,Bicycle,380.50,380.50
1,Canoe,280.00,280.00
2,Parachute,999.99,999.99
3,Unicycle,192.50,180.79


In [30]:
#code
runSELECT(location,'select customerid,count(customerid),sum(price) from itemsordered group by customerid having count(customerid)>1')

,customerid,count(customerid),sum(price)
0,10101,6,320.75
1,10298,5,118.88
2,10299,2,1037.99
3,10330,3,72.75
4,10410,2,281.72
5,10438,3,95.24
6,10439,2,113.50
7,10449,6,930.79


#Order By

In [31]:
#code
runSELECT(location,'select lastname, firstname, city from customers order by lastname;')

,lastname,firstname,city
0,Brown,Leroy,Pinetop
1,Cleaver,Elroy,Globe
2,Dalton,Shawn,Cannon Beach
3,Davids,Donald,Gila Bend
4,Giles,Conrad,Telluride
5,Graham,Sarah,Greensboro
6,Gray,John,Lynden
7,Howell,Michael,Tillamook
8,Howell,Mary Ann,Charleston
9,Jones,Lisa,Oshkosh


In [32]:
#code
runSELECT(location,'select lastname, firstname, city from customers order by lastname desc;')

,lastname,firstname,city
0,Smith,Kevin,Durango
1,Schultz,Ginger,Pocatello
2,Sanchez,Anthony,Winslow
3,Sakahara,Linda,Nogales
4,Moore,Isabela,Yuma
5,Mendoza,Kelly,Kailua
6,Keller,Elroy,Snoqualmie
7,Jones,Lisa,Oshkosh
8,Howell,Michael,Tillamook
9,Howell,Mary Ann,Charleston


In [33]:
#code
runSELECT(location,'select item, price from itemsordered where price >10.00 order by price')

,item,price
0,Ear Muffs,12.50
1,Hoola Hoop,14.75
2,Lantern,16.00
3,Shovel,16.75
4,Rain Coat,18.30
5,Helmet,22.00
6,Pocket Knife,22.38
7,Ski Poles,25.50
8,Pogo stick,28.00
9,Flashlight,28.00


#COMBINING CONDITIONS & BOOLEAN OPERATORS

In [34]:
#code
runSELECT(location,'select customerid,orderdate,item from itemsordered where (item<>"Snow Shoes") and (item<>"Ear Muffs")')

,customerid,orderdate,item
0,10330,1999-06-30,Pogo stick
1,10101,1999-06-30,Raft
2,10298,1999-07-01,Skateboard
3,10101,1999-07-01,Life Vest
4,10299,1999-07-06,Parachute
5,10339,1999-07-27,Umbrella
6,10449,1999-08-13,Unicycle
7,10439,1999-08-14,Ski Poles
8,10101,1999-08-18,Rain Coat
9,10439,1999-09-18,Tent


In [35]:
#code
runSELECT(location,'select item, price from itemsordered where (item like"S%") or (item like"P%") or (item like"F%")')

,item,price
0,Pogo stick,28.00
1,Skateboard,33.00
2,Parachute,999.99
3,Ski Poles,25.50
4,Snow Shoes,45.00
5,Sleeping Bag,89.22
6,Pillow,8.50
7,Flashlight,28.00
8,Flashlight,4.50
9,Sleeping Bag,88.70


# In & Between

In [37]:
#code
runSELECT(location,'select orderdate, item, price from itemsordered where price between 10.00 and 80.00')

,orderdate,item,price
0,1999-06-30,Pogo stick,28.00
1,1999-06-30,Raft,58.00
2,1999-07-01,Skateboard,33.00
3,1999-08-14,Ski Poles,25.50
4,1999-08-18,Rain Coat,18.30
5,1999-09-01,Snow Shoes,45.00
6,1999-09-19,Lantern,29.00
7,1999-12-01,Helmet,22.00
8,1999-12-30,Hoola Hoop,14.75
9,2000-01-01,Flashlight,28.00


In [38]:
#code
runSELECT(location,'select firstname, city, state from customers where state in ("Arizona","Washington","Oklahoma","Colorado","Hawaii")')

,firstname,city,state
0,John,Lynden,Washington
1,Leroy,Pinetop,Arizona
2,Elroy,Snoqualmie,Washington
3,Kelly,Kailua,Hawaii
4,Anthony,Winslow,Arizona
5,Elroy,Globe,Arizona
6,Donald,Gila Bend,Arizona
7,Linda,Nogales,Arizona
8,Kevin,Durango,Colorado
9,Conrad,Telluride,Colorado


#MATHEMATICAL FUNCTIONS

In [39]:
#code
runSELECT(location,'select item,sum(price)/sum(quantity) as unit_price from itemsordered group by item')

,item,unit_price
0,Bicycle,380.500000
1,Canoe,280.000000
2,Canoe paddle,20.000000
3,Compass,8.000000
4,Ear Muffs,12.500000
5,Flashlight,6.500000
6,Helmet,22.000000
7,Hoola Hoop,4.916667
8,Inflatable Mattress,38.000000
9,Lantern,15.000000


#Join

In [47]:
#code
runSELECT(location,'select c.customerid,c.firstname,c.lastname,o.orderdate,o.item,o.price \
from customers c join  itemsordered o on c.customerid = o.customerid order by o.price')

,customerid,firstname,lastname,orderdate,item,price
0,10339,Anthony,Sanchez,1999-07-27,Umbrella,4.50
1,10449,Isabela,Moore,2000-02-29,Flashlight,4.50
2,10438,Kevin,Smith,1999-11-01,Umbrella,6.75
3,10315,Lisa,Jones,2000-02-02,Compass,8.00
4,10438,Kevin,Smith,1999-11-02,Pillow,8.50
5,10298,Leroy,Brown,2000-04-01,Ear Muffs,12.50
6,10101,John,Gray,1999-12-30,Hoola Hoop,14.75
7,10101,John,Gray,2000-01-02,Lantern,16.00
8,10330,Shawn,Dalton,2000-04-19,Shovel,16.75
9,10101,John,Gray,1999-08-18,Rain Coat,18.30


In [50]:
#code
runSELECT(location,'select c.customerid,c.firstname,c.lastname,c.state,o.orderdate,o.item,o.price \
from customers c join  itemsordered o on c.customerid = o.customerid order by c.state desc')

,customerid,firstname,lastname,state,orderdate,item,price
0,10315,Lisa,Jones,Wisconsin,2000-02-02,Compass,8.00
1,10299,Elroy,Keller,Washington,2000-01-18,Inflatable Mattress,38.00
2,10101,John,Gray,Washington,1999-07-01,Life Vest,125.00
3,10101,John,Gray,Washington,1999-06-30,Raft,58.00
4,10101,John,Gray,Washington,1999-08-18,Rain Coat,18.30
5,10299,Elroy,Keller,Washington,1999-07-06,Parachute,999.99
6,10101,John,Gray,Washington,2000-01-02,Lantern,16.00
7,10101,John,Gray,Washington,2000-03-08,Sleeping Bag,88.70
8,10101,John,Gray,Washington,1999-12-30,Hoola Hoop,14.75
9,10410,Mary Ann,Howell,South Carolina,1999-10-28,Sleeping Bag,89.22
